In [3]:
!pip install numpy==2.0.0 -q
!pip install transformers==4.44.0 datasets peft==0.12.0 trl==0.9.6 bitsandbytes accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 106.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.9.6 requires numpy<2.0.0,>=1.18.2, but you have numpy 2.0.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset

In [3]:
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA: {torch.cuda.is_available()}")
print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

✓ PyTorch: 2.9.0+cu126
✓ CUDA: True
✓ GPU: Tesla T4


In [13]:
MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading tokenizer...


In [14]:
print("Loading model (no quantization)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
)


Loading model (no quantization)...


In [15]:
print("Adding LoRA adapters...")
model.enable_input_require_grads()

Adding LoRA adapters...


In [16]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✓ Model ready!")

trainable params: 50,462,720 || all params: 1,150,511,104 || trainable%: 4.3861
✓ Model ready!


In [17]:
filename = "/content/training_data.jsonl"
dataset = load_dataset("json", data_files=filename, split="train")
print(f"✓ Loaded {len(dataset)} examples")

Generating train split: 0 examples [00:00, ? examples/s]

✓ Loaded 1169 examples


In [18]:
def format_example(example):
    msgs = example["messages"]
    text = ""
    for m in msgs:
        role = m["role"]
        content = m["content"]
        if role == "system":
            text += f"<|system|>\n{content}</s>\n"
        elif role == "user":
            text += f"<|user|>\n{content}</s>\n"
        elif role == "assistant":
            text += f"<|assistant|>\n{content}</s>\n"
    return {"text": text}

dataset = dataset.map(format_example)
print(f"Sample:\n{dataset[0]['text'][:400]}")


Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

Sample:
<|system|>
You are Ujjwal. Reply in his casual texting style - short messages, informal language, uses 'ha', 'guru', emojis occasionally.</s>
<|user|>
Done
We wil bedu
On the way</s>
<|assistant|>
Ok</s>



In [33]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=512,
    packing=True,
    args=TrainingArguments(
        output_dir="./outputs",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        warmup_ratio=0.05,
        fp16=True,
        logging_steps=10,
        save_strategy="no",
        optim="adamw_torch",
        report_to="none",
        gradient_checkpointing=True,
    ),
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will overri

In [34]:
print("\n=== TRAINING STARTED ===")
print("This will take ~15-20 minutes on T4...")
trainer.train()
print("\n=== TRAINING COMPLETE ===")


=== TRAINING STARTED ===
This will take ~15-20 minutes on T4...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.628900
20,0.592600
30,0.471800
40,0.418200
50,0.320600
60,0.253900
70,0.201100
80,0.157100
90,0.106600
100,0.087800



=== TRAINING COMPLETE ===


In [35]:
from google.colab import files

model.save_pretrained("ujjwal-lora")
tokenizer.save_pretrained("ujjwal-lora")
print("Model saved!")

!zip -r model.zip ujjwal-lora
files.download("model.zip")

Model saved!
updating: ujjwal-lora/ (stored 0%)
updating: ujjwal-lora/special_tokens_map.json (deflated 73%)
updating: ujjwal-lora/tokenizer.model (deflated 55%)
updating: ujjwal-lora/adapter_config.json (deflated 54%)
updating: ujjwal-lora/tokenizer.json (deflated 74%)
updating: ujjwal-lora/README.md (deflated 66%)
updating: ujjwal-lora/adapter_model.safetensors (deflated 8%)
updating: ujjwal-lora/tokenizer_config.json (deflated 68%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
model.eval()

def chat(msg):
    prompt = f"<|system|>\nYou are Ujjwal. Reply casually.</s>\n<|user|>\n{msg}</s>\n<|assistant|>\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(out[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1].strip()

print("\n=== TESTING ===")
test_msgs = [
    "Raghav: bro did you finish that assignment?",
    "Raghav: want to meet up tomorrow?",
    "Raghav: how was the exam?",
    "Raghav: yo check this out"
]
for q in test_msgs:
    print(f">> {q}")
    print(f"<< {chat(q)}\n")


=== TESTING ===
>> Raghav: bro did you finish that assignment?
<< I am still working

>> Raghav: want to meet up tomorrow?
<< Yes

>> Raghav: how was the exam?
<< All the best and don't forget to drink some alcohol he will ask you 😂

>> Raghav: yo check this out
<< I will let you know

